# PyTorch Geometric Part 1: Message Passing Networks

This notebook covers the foundational message passing networks in Graph Neural Networks (GNNs) with comprehensive mathematical exposition. We'll explore four key architectures that form the backbone of modern GNN research: GCN, GraphSAGE, GIN, and GAT.

## Mathematical Foundation of Graph Neural Networks

### Graph Representation
A graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ consists of:
- **Vertex set** $\mathcal{V} = \{v_1, v_2, \ldots, v_N\}$ where $N = |\mathcal{V}|$ is the number of nodes
- **Edge set** $\mathcal{E} \subseteq \mathcal{V} \times \mathcal{V}$ where each edge $(u,v) \in \mathcal{E}$ connects nodes $u$ and $v$
- **Node features** $\mathbf{X} \in \mathbb{R}^{N \times d}$ where $\mathbf{x}_i \in \mathbb{R}^d$ represents features of node $v_i$
- **Adjacency matrix** $\mathbf{A} \in \{0,1\}^{N \times N}$ where $A_{ij} = 1$ if $(v_i, v_j) \in \mathcal{E}$, else $0$

### Message Passing Framework
The core paradigm of GNNs follows the **Message Passing Neural Network (MPNN)** framework:

$$\mathbf{m}_{u \rightarrow v}^{(l)} = \text{MESSAGE}^{(l)}(\mathbf{h}_u^{(l)}, \mathbf{h}_v^{(l)}, \mathbf{e}_{uv})$$

$$\mathbf{h}_v^{(l+1)} = \text{UPDATE}^{(l)}\left(\mathbf{h}_v^{(l)}, \text{AGGREGATE}^{(l)}\left(\{\mathbf{m}_{u \rightarrow v}^{(l)} : u \in \mathcal{N}(v)\}\right)\right)$$

where:
- $\mathbf{h}_v^{(l)} \in \mathbb{R}^{d^{(l)}}$ is the hidden representation of node $v$ at layer $l$
- $\mathbf{m}_{u \rightarrow v}^{(l)}$ is the message from node $u$ to node $v$ at layer $l$
- $\mathcal{N}(v) = \{u \in \mathcal{V} : (u,v) \in \mathcal{E}\}$ is the neighborhood of node $v$
- $\mathbf{e}_{uv}$ represents optional edge features between nodes $u$ and $v$
- $d^{(l)}$ is the dimensionality of hidden representations at layer $l$

### Theoretical Properties
- **Permutation Invariance**: $f(\pi(\mathbf{X})) = \pi(f(\mathbf{X}))$ for any permutation $\pi$
- **Permutation Equivariance**: Node-level functions preserve node ordering
- **Expressive Power**: Limited by the 1-Weisfeiler-Lehman (1-WL) graph isomorphism test

Message passing enables nodes to aggregate information from their local neighborhoods through learnable functions, with each architecture differing in the specific aggregation mechanism employed.

In [ ]:
# Installation (uncomment if needed)
# !pip install torch torch_geometric torch_scatter torch_sparse torch_cluster torch_spline_conv

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GINConv, GATConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import TUDataset, Planetoid
from torch_geometric.transforms import NormalizeFeatures
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

## 1. Graph Convolutional Networks (GCN)

### Mathematical Formulation

GCN extends the concept of convolution to irregular graph structures through spectral graph theory. The fundamental GCN layer is defined as:

$$\mathbf{H}^{(l+1)} = \sigma\left(\tilde{\mathbf{D}}^{-\frac{1}{2}}\tilde{\mathbf{A}}\tilde{\mathbf{D}}^{-\frac{1}{2}}\mathbf{H}^{(l)}\mathbf{W}^{(l)}\right)$$

**Symbol Definitions:**
- $\mathbf{H}^{(l)} \in \mathbb{R}^{N \times d^{(l)}}$: Node feature matrix at layer $l$
- $\mathbf{W}^{(l)} \in \mathbb{R}^{d^{(l)} \times d^{(l+1)}}$: Learnable weight matrix at layer $l$
- $\tilde{\mathbf{A}} = \mathbf{A} + \mathbf{I}_N$: Adjacency matrix with added self-loops
- $\tilde{\mathbf{D}} \in \mathbb{R}^{N \times N}$: Degree matrix where $\tilde{D}_{ii} = \sum_j \tilde{A}_{ij}$
- $\sigma(\cdot)$: Non-linear activation function (e.g., ReLU, tanh)
- $N$: Number of nodes in the graph

### Spectral Motivation

The GCN formulation is derived from the **localized first-order approximation** of spectral convolutions:

1. **Graph Laplacian**: $\mathbf{L} = \mathbf{D} - \mathbf{A}$ where $\mathbf{D}$ is the degree matrix
2. **Normalized Laplacian**: $\mathcal{L} = \mathbf{I}_N - \mathbf{D}^{-\frac{1}{2}}\mathbf{A}\mathbf{D}^{-\frac{1}{2}}$
3. **Eigendecomposition**: $\mathcal{L} = \mathbf{U}\boldsymbol{\Lambda}\mathbf{U}^T$ where $\mathbf{U}$ contains eigenvectors
4. **Spectral Convolution**: $\mathbf{g}_\theta \star \mathbf{x} = \mathbf{U}\mathbf{g}_\theta(\boldsymbol{\Lambda})\mathbf{U}^T\mathbf{x}$

### Normalization Analysis

The symmetric normalization $\tilde{\mathbf{D}}^{-\frac{1}{2}}\tilde{\mathbf{A}}\tilde{\mathbf{D}}^{-\frac{1}{2}}$ serves multiple purposes:

1. **Prevents vanishing/exploding gradients**: Eigenvalues lie in $[-1, 1]$
2. **Degree-aware aggregation**: High-degree nodes don't dominate
3. **Preserves scale**: Features remain in similar ranges across layers

### Per-Node Update Rule

For each node $v$, the GCN update can be written as:

$$\mathbf{h}_v^{(l+1)} = \sigma\left(\sum_{u \in \mathcal{N}(v) \cup \{v\}} \frac{1}{\sqrt{\tilde{d}_u \tilde{d}_v}} \mathbf{h}_u^{(l)} \mathbf{W}^{(l)}\right)$$

where $\tilde{d}_v = \sum_{u} \tilde{A}_{vu}$ is the degree of node $v$ in $\tilde{\mathbf{A}}$.

GCN's key insight is aggregating information from immediate neighbors using a normalized adjacency matrix, making it the foundational approach for spectral-based graph neural networks.

In [ ]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(GCN, self).__init__()
        self.convs = nn.ModuleList()
        
        # First layer
        self.convs.append(GCNConv(input_dim, hidden_dim))
        
        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        
        # Output layer
        self.convs.append(GCNConv(hidden_dim, output_dim))
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x, edge_index, batch=None):
        # Apply GCN layers
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index)
            x = F.relu(x)
            x = self.dropout(x)
        
        # Final layer without activation
        x = self.convs[-1](x, edge_index)
        
        # For graph classification, pool node features
        if batch is not None:
            x = global_mean_pool(x, batch)
        
        return x

## 2. GraphSAGE (Sample and Aggregate)

### Mathematical Formulation

GraphSAGE addresses scalability and inductive learning through **neighbor sampling** and **learnable aggregation functions**:

$$\mathbf{h}_{\mathcal{N}(v)}^{(l)} = \text{AGGREGATE}_l\left(\{\mathbf{h}_u^{(l)} : u \in \mathcal{S}(\mathcal{N}(v))\}\right)$$

$$\mathbf{h}_v^{(l+1)} = \sigma\left(\mathbf{W}^{(l)} \cdot \text{CONCAT}\left(\mathbf{h}_v^{(l)}, \mathbf{h}_{\mathcal{N}(v)}^{(l)}\right)\right)$$

**Symbol Definitions:**
- $\mathcal{S}(\mathcal{N}(v))$: Sampled subset of neighbors, where $|\mathcal{S}(\mathcal{N}(v))| \leq K$ for fixed sample size $K$
- $\mathbf{h}_{\mathcal{N}(v)}^{(l)} \in \mathbb{R}^{d^{(l)}}$: Aggregated neighborhood representation
- $\text{CONCAT}(\cdot, \cdot)$: Concatenation operation
- $\mathbf{W}^{(l)} \in \mathbb{R}^{d^{(l+1)} \times 2d^{(l)}}$: Weight matrix for concatenated features

### Aggregation Functions

GraphSAGE supports multiple aggregation schemes:

**1. Mean Aggregator:**
$$\text{AGGREGATE}_{\text{mean}} = \frac{1}{|\mathcal{S}(\mathcal{N}(v))|} \sum_{u \in \mathcal{S}(\mathcal{N}(v))} \mathbf{h}_u^{(l)}$$

**2. Max Pooling Aggregator:**
$$\text{AGGREGATE}_{\text{max}} = \max\left(\{\sigma(\mathbf{W}_{\text{pool}}\mathbf{h}_u^{(l)} + \mathbf{b}) : u \in \mathcal{S}(\mathcal{N}(v))\}\right)$$

**3. LSTM Aggregator:**
$$\text{AGGREGATE}_{\text{LSTM}} = \text{LSTM}\left(\{\mathbf{h}_u^{(l)} : u \in \pi(\mathcal{S}(\mathcal{N}(v)))\}\right)$$

where $\pi(\cdot)$ represents a random permutation to make LSTM permutation-invariant.

### Sampling Strategy

**Uniform Sampling:** Select $K$ neighbors uniformly at random:
$$\mathcal{S}(\mathcal{N}(v)) \sim \text{Uniform}(\mathcal{N}(v), K)$$

**Computational Complexity:**
- **Full GCN**: $O(|\mathcal{E}| \cdot d^{(l)} \cdot d^{(l+1)})$ per layer
- **GraphSAGE**: $O(N \cdot K \cdot d^{(l)} \cdot d^{(l+1)})$ per layer

### Inductive Learning Capability

GraphSAGE enables **inductive learning** by learning to aggregate from sampled neighborhoods rather than memorizing specific node embeddings:

1. **Training**: Learn aggregation function on training nodes
2. **Inference**: Apply same function to unseen nodes with their neighbors
3. **Generalization**: No retraining required for new nodes

### L2 Normalization

After each layer update, GraphSAGE applies L2 normalization:
$$\mathbf{h}_v^{(l+1)} = \frac{\mathbf{h}_v^{(l+1)}}{\|\mathbf{h}_v^{(l+1)}\|_2}$$

This normalization helps with:
- **Training stability**: Prevents exploding gradients
- **Clustering properties**: Projects embeddings onto unit sphere
- **Similarity preservation**: Maintains relative distances

GraphSAGE's key innovation is scalable neighbor sampling with learnable aggregation, enabling application to massive graphs while supporting inductive learning for dynamic graph scenarios.

In [ ]:
class GraphSAGE(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(GraphSAGE, self).__init__()
        self.convs = nn.ModuleList()
        
        # First layer
        self.convs.append(SAGEConv(input_dim, hidden_dim))
        
        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_dim, hidden_dim))
        
        # Output layer
        self.convs.append(SAGEConv(hidden_dim, output_dim))
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x, edge_index, batch=None):
        # Apply SAGE layers
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index)
            x = F.relu(x)
            x = self.dropout(x)
        
        # Final layer
        x = self.convs[-1](x, edge_index)
        
        # For graph classification
        if batch is not None:
            x = global_mean_pool(x, batch)
        
        return x

## 3. Graph Isomorphism Networks (GIN)

### Mathematical Formulation

GIN is theoretically motivated by the **Weisfeiler-Lehman (WL) graph isomorphism test** and achieves maximum expressive power among message passing neural networks:

$$\mathbf{h}_v^{(l+1)} = \text{MLP}^{(l)}\left((1 + \epsilon^{(l)}) \cdot \mathbf{h}_v^{(l)} + \sum_{u \in \mathcal{N}(v)} \mathbf{h}_u^{(l)}\right)$$

**Symbol Definitions:**
- $\epsilon^{(l)} \in \mathbb{R}$: Learnable scalar parameter or fixed small constant
- $\text{MLP}^{(l)}$: Multi-layer perceptron at layer $l$
- The sum aggregation ensures permutation invariance

### Theoretical Foundation

**Weisfeiler-Lehman Algorithm:**
The 1-WL algorithm iteratively updates node labels:

1. **Initialize**: $c_v^{(0)} = \mathbf{x}_v$ for each node $v$
2. **Update**: $c_v^{(l+1)} = \text{HASH}\left(c_v^{(l)}, \{\{c_u^{(l)} : u \in \mathcal{N}(v)\}\}\right)$
3. **Multiset**: $\{\{\cdot\}\}$ denotes multiset to preserve duplicate elements

**GIN-WL Correspondence:**
GIN with appropriate MLPs can simulate any function computable by the 1-WL test:

$$\text{If } \mathbf{h}_v^{(l)} = \mathbf{h}_u^{(l)} \text{ then } c_v^{(l)} = c_u^{(l)}$$

### Aggregation Function Analysis

**Sum Aggregation vs. Mean/Max:**

1. **Sum**: $\text{AGG} = \sum_{u \in \mathcal{N}(v)} \mathbf{h}_u^{(l)}$
   - **Injective**: Different multisets → different outputs
   - **WL-equivalent**: Captures structural differences

2. **Mean**: $\text{AGG} = \frac{1}{|\mathcal{N}(v)|} \sum_{u \in \mathcal{N}(v)} \mathbf{h}_u^{(l)}$
   - **Not injective**: Can't distinguish multisets with same mean
   - **Example**: $\{1, 3\}$ and $\{2, 2\}$ both have mean 2

3. **Max**: $\text{AGG} = \max_{u \in \mathcal{N}(v)} \mathbf{h}_u^{(l)}$
   - **Not injective**: Loses information about multiset cardinality
   - **Example**: $\{5\}$ and $\{5, 5, 5\}$ both have max 5

### MLP Architecture

The MLP in GIN typically follows this structure:
$$\text{MLP}^{(l)}(\mathbf{x}) = \mathbf{W}_2^{(l)} \sigma\left(\mathbf{W}_1^{(l)} \mathbf{x} + \mathbf{b}_1^{(l)}\right) + \mathbf{b}_2^{(l)}$$

where:
- $\mathbf{W}_1^{(l)} \in \mathbb{R}^{d^{(l)} \times d^{(l)}}$, $\mathbf{W}_2^{(l)} \in \mathbb{R}^{d^{(l+1)} \times d^{(l)}}$: Weight matrices
- $\mathbf{b}_1^{(l)}, \mathbf{b}_2^{(l)}$: Bias vectors
- $\sigma(\cdot)$: Non-linear activation (ReLU, ELU, etc.)

### Epsilon Parameter

The $\epsilon$ parameter controls self-importance:

- **$\epsilon = 0$**: Node's own features are weighted equally to sum of neighbors
- **$\epsilon > 0$**: Node's own features receive higher weight
- **Learnable $\epsilon$**: Allows model to adapt self-importance per layer

### Graph-Level Representation

For graph classification, GIN combines representations from all layers:
$$\mathbf{h}_G = \text{CONCAT}\left(\text{READOUT}^{(0)}(\{\mathbf{h}_v^{(0)} : v \in \mathcal{V}\}), \ldots, \text{READOUT}^{(L)}(\{\mathbf{h}_v^{(L)} : v \in \mathcal{V}\})\right)$$

where $\text{READOUT}$ can be sum, mean, or max pooling across all nodes.

### Expressive Power Theorem

**Theorem (Xu et al., 2019):** GIN with sufficient MLP capacity is as powerful as the 1-WL test in distinguishing non-isomorphic graphs.

**Proof Sketch:**
1. MLPs with sufficient capacity are universal approximators
2. Sum aggregation preserves multiset information
3. Epsilon parameter enables injective node updates
4. Combination achieves 1-WL expressive power

GIN's key contribution is providing theoretical guarantees on expressive power while maintaining computational efficiency, making it particularly effective for graph classification tasks requiring structural discrimination.

In [ ]:
class GraphGIN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(GraphGIN, self).__init__()
        self.convs = nn.ModuleList()
        
        # First layer with MLP
        mlp1 = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.convs.append(GINConv(mlp1))
        
        # Hidden layers
        for _ in range(num_layers - 2):
            mlp = nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim)
            )
            self.convs.append(GINConv(mlp))
        
        # Output layer
        mlp_out = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        self.convs.append(GINConv(mlp_out))
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x, edge_index, batch=None):
        # Apply GIN layers
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index)
            x = F.relu(x)
            x = self.dropout(x)
        
        # Final layer
        x = self.convs[-1](x, edge_index)
        
        # For graph classification
        if batch is not None:
            x = global_mean_pool(x, batch)
        
        return x

## 4. Graph Attention Networks (GAT)

### Mathematical Formulation

GAT introduces **self-attention mechanisms** to graphs, enabling nodes to dynamically assign different importance weights to different neighbors:

$$\mathbf{h}_v^{(l+1)} = \sigma\left(\sum_{u \in \mathcal{N}(v) \cup \{v\}} \alpha_{vu}^{(l)} \mathbf{W}^{(l)}\mathbf{h}_u^{(l)}\right)$$

**Symbol Definitions:**
- $\alpha_{vu}^{(l)} \in \mathbb{R}$: Attention coefficient from node $u$ to node $v$ at layer $l$
- $\mathbf{W}^{(l)} \in \mathbb{R}^{d^{(l+1)} \times d^{(l)}}$: Shared linear transformation matrix
- $\sum_{u \in \mathcal{N}(v) \cup \{v\}} \alpha_{vu}^{(l)} = 1$: Attention weights sum to 1

### Attention Mechanism

**Step 1: Linear Transformation**
$$\mathbf{h}_u' = \mathbf{W}^{(l)}\mathbf{h}_u^{(l)}$$

**Step 2: Attention Score Computation**
$$e_{vu} = a^{(l)}\left(\mathbf{h}_v', \mathbf{h}_u'\right) = \mathbf{a}^T \text{CONCAT}(\mathbf{h}_v', \mathbf{h}_u')$$

where:
- $\mathbf{a} \in \mathbb{R}^{2d^{(l+1)}}$: Learnable attention parameter vector
- $e_{vu}$: Unnormalized attention score indicating importance of node $u$ to node $v$

**Step 3: Attention Weight Normalization**
$$\alpha_{vu} = \frac{\exp(\text{LeakyReLU}(e_{vu}))}{\sum_{k \in \mathcal{N}(v) \cup \{v\}} \exp(\text{LeakyReLU}(e_{vk}))}$$

The **LeakyReLU** activation and **softmax** normalization ensure:
- Non-linearity in attention computation
- Attention weights sum to 1 across all neighbors
- Numerical stability through exponential normalization

### Multi-Head Attention

GAT employs **multi-head attention** to learn different representation subspaces:

$$\mathbf{h}_v^{(l+1)} = \text{CONCAT}_{k=1}^K \sigma\left(\sum_{u \in \mathcal{N}(v) \cup \{v\}} \alpha_{vu}^{(l,k)} \mathbf{W}^{(l,k)}\mathbf{h}_u^{(l)}\right)$$

**For the final layer (single head with averaging):**
$$\mathbf{h}_v^{(L)} = \sigma\left(\frac{1}{K}\sum_{k=1}^K \sum_{u \in \mathcal{N}(v) \cup \{v\}} \alpha_{vu}^{(L,k)} \mathbf{W}^{(L,k)}\mathbf{h}_u^{(L-1)}\right)$$

where:
- $K$: Number of attention heads
- $\alpha_{vu}^{(l,k)}$: Attention coefficient for head $k$
- $\mathbf{W}^{(l,k)} \in \mathbb{R}^{d^{(l+1)}/K \times d^{(l)}}$: Weight matrix for head $k$

### Attention Computation Details

**Complete Attention Score:**
$$e_{vu} = \text{LeakyReLU}\left(\mathbf{a}^T \begin{bmatrix} \mathbf{W}\mathbf{h}_v \\ \mathbf{W}\mathbf{h}_u \end{bmatrix}\right)$$

**Parametrized as:**
$$e_{vu} = \text{LeakyReLU}\left(\mathbf{a}_1^T \mathbf{W}\mathbf{h}_v + \mathbf{a}_2^T \mathbf{W}\mathbf{h}_u\right)$$

where $\mathbf{a} = [\mathbf{a}_1; \mathbf{a}_2]$ can be decomposed into source and target components.

### Masked Attention

GAT applies **masked attention** to respect graph structure:
- Only compute attention between connected nodes: $(u,v) \in \mathcal{E}$ or $u = v$
- Set $e_{vu} = -\infty$ for non-connected pairs, resulting in $\alpha_{vu} = 0$

### Theoretical Properties

**1. Permutation Equivariance:**
$$\text{GAT}(\pi(\mathbf{X}), \pi(\mathbf{A})\pi^T) = \pi(\text{GAT}(\mathbf{X}, \mathbf{A}))$$

**2. Computational Complexity:**
- **Attention computation**: $O(|\mathcal{E}| \cdot d^{(l+1)})$ per layer
- **Feature transformation**: $O(N \cdot d^{(l)} \cdot d^{(l+1)})$ per layer
- **Total**: $O(|\mathcal{E}| \cdot d^{(l+1)} + N \cdot d^{(l)} \cdot d^{(l+1)})$

**3. Expressive Power:**
GAT with sufficient attention heads can distinguish many graph pairs that simpler GNNs cannot, though it's still limited by the 1-WL hierarchy.

### Attention Interpretation

The attention weights $\alpha_{vu}$ provide **model interpretability**:
- **High $\alpha_{vu}$**: Node $u$ is important for node $v$'s representation
- **Low $\alpha_{vu}$**: Node $u$ has minimal influence on node $v$
- **Attention patterns**: Can reveal graph structures (hubs, communities, etc.)

### Variants and Extensions

**1. Graph Attention Networks v2 (GATv2):**
$$e_{vu} = \mathbf{a}^T \text{LeakyReLU}(\mathbf{W}[\mathbf{h}_v; \mathbf{h}_u])$$

**2. Graph Transformer:**
Incorporates positional encodings and full self-attention across all node pairs.

GAT's key innovation is learning adaptive attention weights that capture task-relevant node relationships, providing both improved performance and model interpretability through attention visualization.

In [ ]:
class GraphGAT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, heads=4):
        super(GraphGAT, self).__init__()
        self.convs = nn.ModuleList()
        
        # First layer with multi-head attention
        self.convs.append(GATConv(input_dim, hidden_dim, heads=heads, dropout=0.1))
        
        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_dim * heads, hidden_dim, heads=heads, dropout=0.1))
        
        # Output layer (single head)
        self.convs.append(GATConv(hidden_dim * heads, output_dim, heads=1, dropout=0.1))
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x, edge_index, batch=None):
        # Apply GAT layers
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index)
            x = F.elu(x)
            x = self.dropout(x)
        
        # Final layer
        x = self.convs[-1](x, edge_index)
        
        # For graph classification
        if batch is not None:
            x = global_mean_pool(x, batch)
        
        return x

## 5. Dataset Preparation

### Cora Citation Network Dataset

We'll use the **Cora dataset** for node classification, a canonical benchmark in graph neural networks:

**Dataset Characteristics:**
- **Domain**: Academic paper citation network
- **Task**: Multi-class node classification (7 classes)
- **Node Features**: Bag-of-words representation (1433 dimensions)
- **Graph Structure**: Citation links between papers
- **Classes**: Theory, Neural Networks, Probabilistic Methods, Genetic Algorithms, Case Based, Reinforcement Learning, Rule Learning

**Mathematical Representation:**
- **Graph**: $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ where $|\mathcal{V}| = 2708$ nodes, $|\mathcal{E}| = 10556$ edges
- **Feature Matrix**: $\mathbf{X} \in \{0,1\}^{2708 \times 1433}$ (binary bag-of-words)
- **Adjacency Matrix**: $\mathbf{A} \in \{0,1\}^{2708 \times 2708}$ (undirected, sparse)
- **Labels**: $\mathbf{y} \in \{0,1,2,3,4,5,6\}^{2708}$

**Data Splits:**
- **Training**: 140 labeled nodes (20 per class)
- **Validation**: 500 nodes for hyperparameter tuning
- **Test**: 1000 nodes for final evaluation
- **Semi-supervised Setting**: Only training nodes have labels during training

**Graph Properties:**
- **Average Degree**: $\bar{d} = \frac{2|\mathcal{E}|}{|\mathcal{V}|} = \frac{2 \times 10556}{2708} \approx 7.8$
- **Clustering Coefficient**: Measures local connectivity density
- **Diameter**: Maximum shortest path between any two connected nodes
- **Homophily**: Tendency for connected nodes to have similar labels

We'll demonstrate how different message passing architectures handle this semi-supervised node classification task.

In [ ]:
# Load Cora dataset for node classification
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

print(f'Dataset: {dataset}')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

## 6. Training Function

### Mathematical Foundation of GNN Training

**Loss Function for Node Classification:**
$$\mathcal{L} = -\frac{1}{|\mathcal{V}_{\text{train}}|} \sum_{v \in \mathcal{V}_{\text{train}}} \sum_{c=1}^C y_{vc} \log(\hat{y}_{vc})$$

where:
- $\mathcal{V}_{\text{train}}$: Set of training nodes (labeled nodes)
- $C$: Number of classes (7 for Cora)
- $y_{vc} \in \{0,1\}$: True label (one-hot encoded)
- $\hat{y}_{vc} = \text{softmax}(\mathbf{z}_v)_c$: Predicted probability for class $c$
- $\mathbf{z}_v$: Logits from GNN for node $v$

**Gradient Computation:**
$$\frac{\partial \mathcal{L}}{\partial \theta} = \frac{\partial \mathcal{L}}{\partial \mathbf{Z}} \cdot \frac{\partial \mathbf{Z}}{\partial \mathbf{H}^{(L)}} \cdots \frac{\partial \mathbf{H}^{(1)}}{\partial \mathbf{H}^{(0)}} \cdot \frac{\partial \mathbf{H}^{(0)}}{\partial \theta}$$

where $\theta$ represents all learnable parameters and backpropagation follows the message passing chain.

**Optimization Algorithm:**
We use **Adam optimizer** with the update rule:
$$\theta_{t+1} = \theta_t - \frac{\alpha}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t$$

where:
- $\hat{m}_t = \frac{m_t}{1-\beta_1^t}$: Bias-corrected first moment estimate
- $\hat{v}_t = \frac{v_t}{1-\beta_2^t}$: Bias-corrected second moment estimate
- $\alpha$: Learning rate, $\beta_1, \beta_2$: Exponential decay rates
- $\epsilon$: Small constant for numerical stability

**Regularization:**
- **Dropout**: $\mathbf{h}_v^{(l)} \sim \text{Bernoulli}(p) \odot \mathbf{h}_v^{(l)} / p$ during training
- **Weight Decay**: $\mathcal{L}_{\text{total}} = \mathcal{L} + \lambda \sum_i \|\mathbf{W}_i\|_2^2$

**Evaluation Metrics:**
- **Accuracy**: $\text{Acc} = \frac{1}{|\mathcal{V}_{\text{test}}|} \sum_{v \in \mathcal{V}_{\text{test}}} \mathbb{I}[\arg\max_c \hat{y}_{vc} = \arg\max_c y_{vc}]$
- **Cross-Entropy Loss**: Used for monitoring training progress

The unified training function enables fair comparison across different GNN architectures under identical optimization conditions.

In [ ]:
def train_model(model, data, epochs=200, lr=0.01):
    """
    Train a GNN model on node classification task
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    data = data.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    train_losses = []
    val_accuracies = []
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        # Validation accuracy
        if epoch % 20 == 0:
            model.eval()
            with torch.no_grad():
                pred = model(data.x, data.edge_index).argmax(dim=1)
                val_acc = accuracy_score(data.y[data.val_mask].cpu(), pred[data.val_mask].cpu())
                val_accuracies.append(val_acc)
                print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}')
            model.train()
    
    return train_losses, val_accuracies

def test_model(model, data):
    """
    Test a trained GNN model
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    data = data.to(device)
    
    model.eval()
    with torch.no_grad():
        pred = model(data.x, data.edge_index).argmax(dim=1)
        test_acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    
    return test_acc

## 7. Model Comparison

### Theoretical Comparison Framework

We'll conduct a **controlled experiment** comparing the four message passing architectures under identical conditions:

**Experimental Controls:**
- **Hidden Dimensions**: $d^{(1)} = 64$ for all models (except GAT: $16 \times 4$ heads)
- **Number of Layers**: $L = 2$ layers for fair comparison
- **Optimizer**: Adam with $\alpha = 0.01$, $\beta_1 = 0.9$, $\beta_2 = 0.999$
- **Regularization**: Dropout $p = 0.5$, Weight decay $\lambda = 5 \times 10^{-4}$
- **Training Epochs**: 100 iterations
- **Random Seed**: Fixed for reproducibility

**Model Parameter Comparison:**

1. **GCN**: $\theta_{\text{GCN}} = \{\mathbf{W}^{(1)} \in \mathbb{R}^{1433 \times 64}, \mathbf{W}^{(2)} \in \mathbb{R}^{64 \times 7}\}$
   - **Parameters**: $1433 \times 64 + 64 \times 7 = 92,160$

2. **GraphSAGE**: $\theta_{\text{SAGE}} = \{\mathbf{W}^{(1)} \in \mathbb{R}^{2866 \times 64}, \mathbf{W}^{(2)} \in \mathbb{R}^{128 \times 7}\}$
   - **Parameters**: $2866 \times 64 + 128 \times 7 = 184,320$ (due to concatenation)

3. **GIN**: $\theta_{\text{GIN}} = \{\text{MLP}_1, \text{MLP}_2\}$ where each MLP has 2 layers
   - **Parameters**: $\approx 200,000$ (due to MLPs in each layer)

4. **GAT**: $\theta_{\text{GAT}} = \{\mathbf{W}^{(1,k)}, \mathbf{a}^{(1,k)}\}_{k=1}^4$ for 4 heads
   - **Parameters**: $4 \times (1433 \times 16 + 32) + 64 \times 7 = 92,160$

**Expected Performance Characteristics:**

- **GCN**: Baseline performance, good for homophilic graphs
- **GraphSAGE**: May handle heterophily better due to concatenation
- **GIN**: Strong structural discrimination, good for complex patterns
- **GAT**: Adaptive attention should focus on relevant neighbors

**Computational Complexity per Forward Pass:**
- **GCN**: $O(|\mathcal{E}| \cdot d + N \cdot d^2)$
- **GraphSAGE**: $O(N \cdot K \cdot d^2)$ with sampling size $K$
- **GIN**: $O(|\mathcal{E}| \cdot d + N \cdot d^2)$ plus MLP overhead
- **GAT**: $O(|\mathcal{E}| \cdot d + N \cdot d^2)$ plus attention computation

This controlled comparison will reveal how different aggregation mechanisms perform on the semi-supervised node classification task.

In [ ]:
# Initialize models
models = {
    'GCN': GCN(dataset.num_features, 64, dataset.num_classes),
    'GraphSAGE': GraphSAGE(dataset.num_features, 64, dataset.num_classes),
    'GIN': GraphGIN(dataset.num_features, 64, dataset.num_classes),
    'GAT': GraphGAT(dataset.num_features, 16, dataset.num_classes, heads=4)  # Smaller hidden dim due to multi-head
}

# Train and evaluate each model
results = {}

for name, model in models.items():
    print(f"\n=== Training {name} ===")
    train_losses, val_accs = train_model(model, data, epochs=100)
    test_acc = test_model(model, data)
    
    results[name] = {
        'model': model,
        'train_losses': train_losses,
        'val_accuracies': val_accs,
        'test_accuracy': test_acc
    }
    
    print(f"Final Test Accuracy for {name}: {test_acc:.4f}")

## 8. Visualization and Analysis

### Mathematical Foundation of Embedding Analysis

**t-Distributed Stochastic Neighbor Embedding (t-SNE):**

t-SNE reduces high-dimensional embeddings $\mathbf{h}_v \in \mathbb{R}^{d}$ to 2D visualizations while preserving local neighborhood structure.

**Step 1: Compute High-Dimensional Similarities**
$$p_{j|i} = \frac{\exp(-\|\mathbf{h}_i - \mathbf{h}_j\|^2 / 2\sigma_i^2)}{\sum_{k \neq i} \exp(-\|\mathbf{h}_i - \mathbf{h}_k\|^2 / 2\sigma_i^2)}$$

$$p_{ij} = \frac{p_{j|i} + p_{i|j}}{2N}$$

where $\sigma_i$ is chosen such that perplexity $\text{Perp}(P_i) = 2^{H(P_i)}$ equals desired value.

**Step 2: Compute Low-Dimensional Similarities**
$$q_{ij} = \frac{(1 + \|\mathbf{y}_i - \mathbf{y}_j\|^2)^{-1}}{\sum_{k \neq l} (1 + \|\mathbf{y}_k - \mathbf{y}_l\|^2)^{-1}}$$

where $\mathbf{y}_i \in \mathbb{R}^2$ is the 2D embedding of node $i$.

**Step 3: Minimize KL Divergence**
$$\mathcal{L} = \text{KL}(P||Q) = \sum_i \sum_j p_{ij} \log \frac{p_{ij}}{q_{ij}}$$

**Interpretation of Visualizations:**

1. **Cluster Quality**: Nodes of same class should form tight clusters
2. **Separation**: Different classes should be well-separated
3. **Local Structure**: Nearby nodes in 2D should be similar in original space
4. **Global Structure**: Overall arrangement should reflect graph topology

**Embedding Quality Metrics:**

**1. Silhouette Score:**
$$s_i = \frac{b_i - a_i}{\max(a_i, b_i)}$$

where:
- $a_i$: Average distance to nodes in same cluster
- $b_i$: Average distance to nodes in nearest different cluster
- $s_i \in [-1, 1]$: Higher values indicate better clustering

**2. Homophily Analysis:**
$$H = \frac{1}{|\mathcal{E}|} \sum_{(u,v) \in \mathcal{E}} \mathbb{I}[y_u = y_v]$$

Measures fraction of edges connecting nodes with same labels.

**3. Modularity:**
$$Q = \frac{1}{2m} \sum_{ij} \left(A_{ij} - \frac{k_i k_j}{2m}\right) \delta(c_i, c_j)$$

where $k_i$ is degree of node $i$, $m = |\mathcal{E}|$, and $\delta(c_i, c_j) = 1$ if nodes $i,j$ are in same community.

Training curve analysis reveals convergence behavior and potential overfitting through validation loss monitoring.

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Training loss
for name, result in results.items():
    axes[0].plot(result['train_losses'], label=name, alpha=0.8)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Training Loss')
axes[0].set_title('Training Loss Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Test accuracy comparison
names = list(results.keys())
test_accs = [results[name]['test_accuracy'] for name in names]

bars = axes[1].bar(names, test_accs, alpha=0.8, color=['skyblue', 'lightcoral', 'lightgreen', 'orange'])
axes[1].set_ylabel('Test Accuracy')
axes[1].set_title('Test Accuracy Comparison')
axes[1].set_ylim(0, 1)

# Add value labels on bars
for bar, acc in zip(bars, test_accs):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{acc:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 9. Node Embeddings Visualization

### Mathematical Analysis of Learned Representations

**Embedding Space Properties:**

The learned node embeddings $\mathbf{h}_v^{(L)} \in \mathbb{R}^{d^{(L)}}$ should exhibit several desirable properties:

**1. Class Separation:**
$$\mathbb{E}_{u,v: y_u = y_v}[\|\mathbf{h}_u - \mathbf{h}_v\|_2] < \mathbb{E}_{u,v: y_u \neq y_v}[\|\mathbf{h}_u - \mathbf{h}_v\|_2]$$

Intra-class distances should be smaller than inter-class distances.

**2. Neighborhood Preservation:**
$$\text{sim}(\mathbf{h}_u, \mathbf{h}_v) \propto \text{graph\_distance}(u, v)^{-1}$$

Nearby nodes in the graph should have similar embeddings.

**3. Linear Separability:**
$$\exists \mathbf{w}, b : \text{sign}(\mathbf{w}^T\mathbf{h}_v + b) = y_v \text{ for most nodes } v$$

Classes should be linearly separable in embedding space.

**Dimensionality Analysis:**

The embedding dimensionality $d^{(L)}$ affects representation capacity:

- **Low $d^{(L)}$**: May underfit, insufficient representation power
- **High $d^{(L)}$**: May overfit, computational overhead
- **Optimal $d^{(L)}$**: Balances expressiveness and generalization

**Intrinsic Dimensionality Estimation:**
$$d_{\text{intrinsic}} \approx -\frac{\log(N)}{\log(\epsilon)} \text{ where } \epsilon \text{ covers most points}$$

**Embedding Quality Assessment:**

**1. Neighborhood Preservation:**
$$R_{k}(v) = \frac{|N_k^{\text{graph}}(v) \cap N_k^{\text{embedding}}(v)|}{k}$$

where $N_k^{\text{graph}}(v)$ are $k$-nearest neighbors in graph, $N_k^{\text{embedding}}(v)$ in embedding space.

**2. Trustworthiness:**
$$T(k) = 1 - \frac{2}{Nk(2N-3k-1)} \sum_{v=1}^N \sum_{u \in U_k(v)} (r(v,u) - k)$$

where $U_k(v)$ are false neighbors and $r(v,u)$ is rank of $u$ in graph neighborhood of $v$.

**Class Distribution in Embedding Space:**

For each class $c$, we can analyze the embedding distribution:
$$\boldsymbol{\mu}_c = \frac{1}{|V_c|} \sum_{v \in V_c} \mathbf{h}_v$$
$$\boldsymbol{\Sigma}_c = \frac{1}{|V_c|} \sum_{v \in V_c} (\mathbf{h}_v - \boldsymbol{\mu}_c)(\mathbf{h}_v - \boldsymbol{\mu}_c)^T$$

The visualization should reveal whether classes form compact, well-separated clusters in the learned embedding space.

In [ ]:
# Get embeddings from the best model (let's use GCN as an example)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = results['GCN']['model'].to(device)
data = data.to(device)

model.eval()
with torch.no_grad():
    # Get embeddings from the second-to-last layer
    x = data.x
    for conv in model.convs[:-1]:
        x = conv(x, data.edge_index)
        x = F.relu(x)
        x = model.dropout(x)
    
    embeddings = x.cpu().numpy()
    labels = data.y.cpu().numpy()

# Apply t-SNE for visualization
print("Applying t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
embeddings_2d = tsne.fit_transform(embeddings)

# Plot embeddings
plt.figure(figsize=(10, 8))
scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                     c=labels, cmap='tab10', alpha=0.7, s=20)
plt.colorbar(scatter, label='Class')
plt.title('Node Embeddings Visualization (GCN)')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Embedding shape: {embeddings.shape}")
print(f"Number of classes: {len(np.unique(labels))}")

## 10. Architecture Comparison Summary

### Comprehensive Mathematical Analysis

**Performance Comparison Framework:**

We evaluate models across multiple dimensions using rigorous statistical analysis:

**1. Predictive Performance:**
- **Test Accuracy**: $\text{Acc} = \frac{|\{v \in \mathcal{V}_{\text{test}} : \hat{y}_v = y_v\}|}{|\mathcal{V}_{\text{test}}|}$
- **F1-Score**: $F_1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$
- **Area Under ROC**: $\text{AUC} = \int_0^1 \text{TPR}(\text{FPR}^{-1}(x)) dx$

**2. Model Complexity:**
- **Parameter Count**: $|\theta| = \sum_l |\mathbf{W}^{(l)}| + |\mathbf{b}^{(l)}|$
- **FLOPs per Forward Pass**: Floating-point operations required
- **Memory Consumption**: Peak GPU/CPU memory usage

**3. Computational Efficiency:**
- **Training Time**: Wall-clock time per epoch
- **Inference Latency**: Time for single forward pass
- **Scalability**: Performance vs. graph size relationship

**Theoretical Comparison:**

| Architecture | Aggregation | Complexity | Expressive Power | Scalability |
|-------------|-------------|-------------|------------------|-------------|
| **GCN** | $\tilde{\mathbf{D}}^{-1/2}\tilde{\mathbf{A}}\tilde{\mathbf{D}}^{-1/2}$ | $O(\|\mathcal{E}\|d)$ | 1-WL limited | Full graph |
| **GraphSAGE** | Sampling + Concat | $O(NKd)$ | 1-WL limited | Subgraph |
| **GIN** | Sum + MLP | $O(\|\mathcal{E}\|d)$ | **1-WL equivalent** | Full graph |
| **GAT** | Attention weights | $O(\|\mathcal{E}\|d)$ | Beyond 1-WL | Full graph |

**Symbol Definitions:**
- $d$: Hidden dimension
- $K$: Sample size (GraphSAGE)
- $N$: Number of nodes
- $|\mathcal{E}|$: Number of edges

**Statistical Significance Testing:**

For comparing model performances, we use:

**Paired t-test:**
$$t = \frac{\bar{d}}{\frac{s_d}{\sqrt{n}}}$$

where $\bar{d}$ is mean difference, $s_d$ is standard deviation of differences, $n$ is number of trials.

**Effect Size (Cohen's d):**
$$d = \frac{\mu_1 - \mu_2}{\sigma_{\text{pooled}}}$$

**Confidence Intervals:**
$$CI = \bar{x} \pm t_{\alpha/2,n-1} \frac{s}{\sqrt{n}}$$

**Key Insights:**

1. **GCN**: Establishes strong baseline through spectral normalization
2. **GraphSAGE**: Trades some accuracy for scalability via sampling
3. **GIN**: Provides theoretical guarantees on expressive power
4. **GAT**: Offers interpretability through attention weights

**Architecture Selection Guidelines:**

- **Small-Medium Graphs + High Accuracy**: GIN or GAT
- **Large Graphs + Scalability**: GraphSAGE
- **Interpretability Required**: GAT (attention visualization)
- **Baseline/Simple Implementation**: GCN
- **Theoretical Guarantees**: GIN (WL-test equivalence)

The comparison reveals trade-offs between expressiveness, computational efficiency, and theoretical properties.

In [ ]:
# Model complexity comparison
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("=== Model Comparison Summary ===")
print(f"{'Architecture':<12} {'Test Acc':<10} {'Parameters':<12} {'Key Features'}")
print("-" * 70)

architecture_info = {
    'GCN': 'Normalized adjacency, spectral approach',
    'GraphSAGE': 'Sampling-based, inductive learning',
    'GIN': 'Theoretically powerful, graph isomorphism',
    'GAT': 'Attention mechanism, dynamic weighting'
}

for name, result in results.items():
    test_acc = result['test_accuracy']
    n_params = count_parameters(result['model'])
    features = architecture_info[name]
    print(f"{name:<12} {test_acc:<10.4f} {n_params:<12} {features}")

# Find best performing model
best_model = max(results.keys(), key=lambda x: results[x]['test_accuracy'])
print(f"\nBest performing model: {best_model} with {results[best_model]['test_accuracy']:.4f} test accuracy")

print("\n=== Architecture Insights ===")
print("• GCN: Simple and effective, good baseline")
print("• GraphSAGE: Scalable to large graphs, handles new nodes")
print("• GIN: Strong theoretical foundation, good for graph classification")
print("• GAT: Attention provides interpretability and adaptive aggregation")

## 11. Practical Tips for CPU Usage

### Mathematical Optimization for Resource-Constrained Environments

**Computational Complexity Analysis for M2 MacBook Air:**

The M2 chip has specific characteristics that affect GNN performance:
- **CPU Cores**: 8 cores (4 performance + 4 efficiency)
- **Memory Bandwidth**: Unified memory architecture
- **SIMD Instructions**: Advanced vector operations support

**Model Size Optimization:**

**1. Parameter Reduction Strategies:**

**Hidden Dimension Scaling:**
$$\text{Parameters} \propto d_{\text{input}} \times d_{\text{hidden}} + d_{\text{hidden}}^2 \times L$$

Optimal hidden dimension follows:
$$d_{\text{optimal}} = \sqrt{\frac{\text{Data Complexity}}{\text{Model Capacity}}}$$

**Layer Depth Analysis:**
$$\text{Receptive Field} = L \text{ hops in graph}$$
$$\text{Over-smoothing Risk} \propto e^{-L/\tau}$$

where $\tau$ is the characteristic smoothing length scale.

**2. Memory Complexity:**

**Forward Pass Memory:**
$$M_{\text{forward}} = N \times d^{(l)} \times L + |\mathcal{E}| \times \text{edge\_features}$$

**Gradient Memory:**
$$M_{\text{gradient}} = 2 \times M_{\text{forward}} + \text{Parameter Storage}$$

**Memory-Efficient Techniques:**

**Gradient Checkpointing:**
$$M_{\text{total}} = M_{\text{activations}} + \sqrt{L} \times M_{\text{checkpoint}}$$

**Mixed Precision Training:**
$$\text{Memory Reduction} \approx 50\% \text{ with minimal accuracy loss}$$

**3. Computational Optimization:**

**Sparse Matrix Operations:**
For adjacency matrix $\mathbf{A}$ with sparsity $\rho = \frac{|\mathcal{E}|}{N^2}$:
$$\text{Dense Complexity}: O(N^2 d)$$
$$\text{Sparse Complexity}: O(|\mathcal{E}| d) = O(\rho N^2 d)$$

**Batch Processing:**
$$\text{Throughput} = \frac{\text{Batch Size}}{\text{Processing Time}} \times \text{CPU Utilization}$$

**Thread Optimization:**
For M2 MacBook Air, optimal thread count:
$$N_{\text{threads}} = \min(8, \text{available\_cores})$$

**PyTorch-Specific Optimizations:**

```python
# Optimal thread configuration
torch.set_num_threads(8)  # Use all 8 cores
torch.set_num_interop_threads(2)  # Reduce overhead

# Memory optimization
torch.backends.cudnn.benchmark = False  # Disable for CPU
torch.backends.mkldnn.enabled = True   # Enable Intel MKL-DNN
```

**Profiling and Monitoring:**

**Memory Usage:**
$$\text{Peak Memory} = \max_t \sum_{i} \text{tensor\_size}_i(t)$$

**CPU Utilization:**
$$\text{Efficiency} = \frac{\text{Actual FLOPS}}{\text{Theoretical Peak FLOPS}}$$

These optimizations ensure efficient GNN training on resource-constrained CPU environments while maintaining model quality.

In [ ]:
# CPU Optimization Tips
print("=== CPU Optimization Tips for GNNs ===")
print("\n1. Reduce model complexity:")
print("   - Use fewer layers (2-3 is often sufficient)")
print("   - Reduce hidden dimensions (32-64 instead of 128+)")
print("   - Use fewer attention heads in GAT (2-4 instead of 8)")

print("\n2. Efficient data handling:")
print("   - Use smaller batch sizes for graph-level tasks")
print("   - Enable torch.set_num_threads() for CPU parallelization")
print("   - Consider data preprocessing to reduce graph size")

print("\n3. Memory management:")
print("   - Use gradient accumulation for large graphs")
print("   - Clear cache regularly with torch.cuda.empty_cache() (even for CPU)")
print("   - Use mixed precision when available")

# Set optimal number of threads for M2 MacBook Air
torch.set_num_threads(8)  # M2 has 8 cores
print(f"\nCurrent PyTorch threads: {torch.get_num_threads()}")
print(f"Using device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

## Summary

### Comprehensive Mathematical Foundation of Message Passing Networks

In this notebook, we have explored the mathematical foundations and practical implementations of four fundamental message passing architectures:

### **1. Graph Convolutional Networks (GCN)**
- **Mathematical Core**: $\mathbf{H}^{(l+1)} = \sigma\left(\tilde{\mathbf{D}}^{-\frac{1}{2}}\tilde{\mathbf{A}}\tilde{\mathbf{D}}^{-\frac{1}{2}}\mathbf{H}^{(l)}\mathbf{W}^{(l)}\right)$
- **Key Innovation**: Spectral approach with symmetric normalization
- **Strengths**: Simple, theoretically grounded, good baseline performance
- **Limitations**: Fixed aggregation, requires full graph access

### **2. GraphSAGE (Sample and Aggregate)**
- **Mathematical Core**: $\mathbf{h}_v^{(l+1)} = \sigma\left(\mathbf{W}^{(l)} \cdot [\mathbf{h}_v^{(l)} || \text{AGG}(\{\mathbf{h}_u^{(l)} : u \in \mathcal{S}(\mathcal{N}(v))\})]\right)$
- **Key Innovation**: Neighbor sampling for scalability and inductive learning
- **Strengths**: Scalable, handles new nodes, multiple aggregators
- **Limitations**: Sampling introduces variance, increased parameters

### **3. Graph Isomorphism Networks (GIN)**
- **Mathematical Core**: $\mathbf{h}_v^{(l+1)} = \text{MLP}^{(l)}\left((1 + \epsilon^{(l)}) \cdot \mathbf{h}_v^{(l)} + \sum_{u \in \mathcal{N}(v)} \mathbf{h}_u^{(l)}\right)$
- **Key Innovation**: Theoretical equivalence to Weisfeiler-Lehman test
- **Strengths**: Maximum expressive power among MPNNs, principled design
- **Limitations**: More parameters due to MLPs, potential overfitting

### **4. Graph Attention Networks (GAT)**
- **Mathematical Core**: $\mathbf{h}_v^{(l+1)} = \sigma\left(\sum_{u \in \mathcal{N}(v) \cup \{v\}} \alpha_{vu}^{(l)} \mathbf{W}^{(l)}\mathbf{h}_u^{(l)}\right)$
- **Key Innovation**: Dynamic attention-based neighbor weighting
- **Strengths**: Adaptive aggregation, interpretable attention, multi-head capability
- **Limitations**: Increased computational cost, attention may not always help

### **Theoretical Insights**

**Expressive Power Hierarchy:**
$$\text{GAT} \supseteq \text{GIN} \equiv \text{1-WL} \supseteq \text{GraphSAGE} \approx \text{GCN}$$

**Computational Complexity Comparison:**
- **Memory**: $O(N \times d \times L + |\theta|)$ for all architectures
- **Time**: $O(|\mathcal{E}| \times d^2)$ for full-graph methods, $O(N \times K \times d^2)$ for sampling
- **Scalability**: GraphSAGE > GCN ≈ GIN ≈ GAT

**Practical Guidelines:**

**Architecture Selection Matrix:**
| Use Case | Recommended Architecture | Rationale |
|----------|-------------------------|-----------|
| **Small graphs, high accuracy** | GIN or GAT | Maximum expressive power |
| **Large graphs, scalability** | GraphSAGE | Sublinear complexity via sampling |
| **Interpretability needed** | GAT | Attention weight visualization |
| **Simple baseline** | GCN | Well-established, reliable |
| **Theoretical guarantees** | GIN | WL-test equivalence |

**Key Mathematical Principles:**
1. **Permutation Invariance**: All architectures respect graph symmetries
2. **Locality**: Information propagates through graph structure
3. **Learnable Aggregation**: Different mechanisms for combining neighbor information
4. **Depth vs. Over-smoothing**: Trade-off between receptive field and feature distinguishability

### **Next Steps**

The next notebook will cover **graph autoencoders** for unsupervised learning, exploring:
- **Variational Graph Autoencoders (VGAE)**: $\mathcal{L} = -\mathbb{E}_{q(\mathbf{Z}|\mathbf{X},\mathbf{A})}[\log p(\mathbf{A}|\mathbf{Z})] + \text{KL}[q(\mathbf{Z}|\mathbf{X},\mathbf{A})||p(\mathbf{Z})]$
- **Graph Generation**: Learning to generate new graph structures
- **Link Prediction**: Predicting missing edges in networks
- **Node Clustering**: Unsupervised community detection

This foundation in message passing networks provides the mathematical framework necessary for understanding advanced graph neural network architectures and their applications.